In [2]:
import com.toldoven.aoc.notebook.AocClient

val aoc = AocClient.fromEnv().interactiveDay(2024, 2)

aoc.viewPartOne()

00:44:11,049 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Could NOT find resource [logback-test.xml]
00:44:11,049 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Could NOT find resource [logback.groovy]
00:44:11,049 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Found resource [logback.xml] at [jar:file:/C:/Users/hextanium/AppData/Local/JetBrains/IntelliJIdea2024.1/kotlinNotebook/aoc-2024.1052ec5d/kernels/0.12.0-184/kotlin-jupyter-kernel-shadowed-jar/kotlin-jupyter-kernel-shadowed-0.12.0-184.jar!/logback.xml]
00:44:11,050 |-WARN in ch.qos.logback.classic.LoggerContext[default] - Resource [logback.xml] occurs multiple times on the classpath.
00:44:11,050 |-WARN in ch.qos.logback.classic.LoggerContext[default] - Resource [logback.xml] occurs at [jar:file:/C:/Users/hextanium/.gradle/caches/modules-2/files-2.1/org.jetbrains.kotlinx/kotlin-jupyter-kernel/0.12.0-339/78f122af112bc1cbfa63050b6b2114107ac14495/kotlin-jupyter-kernel-0.12.0-339.jar!/logback.xml]
0

--- Day 2: Red-Nosed Reports --- 
 Fortunately, the first location The Historians want to search isn't a long walk from the Chief Historian's office. 
 While the Red-Nosed Reindeer nuclear fusion/fission plant appears to contain no sign of the Chief Historian, the engineers there run up to you as soon as they see you. Apparently, they still talk about the time Rudolph was saved through molecular synthesis from a single electron. 
 They're quick to add that - since you're already here - they'd really appreciate your help analyzing some unusual data from the Red-Nosed reactor. You turn to check if The Historians are waiting for you, but they seem to have already divided into groups that are currently searching every corner of the facility. You offer to help with the unusual data. 
 The unusual data (your puzzle input) consists of many reports , one report per line. Each report is a list of numbers called levels that are separated by spaces. For example: 
 7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
 
 This example data contains six reports each containing five levels. 
 The engineers are trying to figure out which reports are safe . The Red-Nosed reactor safety systems can only tolerate levels that are either gradually increasing or gradually decreasing. So, a report only counts as safe if both of the following are true: 
 
 The levels are either all increasing or all decreasing . 
 Any two adjacent levels differ by at least one and at most three . 
 
 In the example above, the reports can be found safe or unsafe by checking those rules: 
 
 7 6 4 2 1 : Safe because the levels are all decreasing by 1 or 2. 
 1 2 7 8 9 : Unsafe because 2 7 is an increase of 5. 
 9 7 6 2 1 : Unsafe because 6 2 is a decrease of 4. 
 1 3 2 4 5 : Unsafe because 1 3 is increasing but 3 2 is decreasing. 
 8 6 4 4 1 : Unsafe because 4 4 is neither an increase or a decrease. 
 1 3 6 7 9 : Safe because the levels are all increasing by 1, 2, or 3. 
 
 So, in this example, 2 reports are safe . 
 Analyze the unusual data from the engineers. How many reports are safe?

In [6]:
fun delta(pair: Pair<Int, Int>) = pair.first - pair.second
fun neighboredDelta(report: List<Int>) = report.zipWithNext().map(::delta)
fun List<List<Int>>.filterInRange(lower: Int, upper: Int) = this.filter { it.all { abs(it) in lower..upper } }

aoc.input()
    .lines()
    .map { it.split(" ") }
    .map { it.map { it.toInt() } }
    .filter { (it.sorted() == it) or (it.sortedDescending() == it) } // check if levels are all increasing or decreasing
    .map { (neighboredDelta(it) to neighboredDelta(it.reversed())) } // get the deltas in the forward and backward direction
    .unzip() // unzip so i can deal with all of the forward and all of the backwards independently
    .let { (forward, backward) -> forward.filterInRange(1, 3) to backward.filterInRange(1, 3) } // check if the deltas are in the given range  
    .let { (forward, backward) -> min(forward.size, backward.size) } // get the least number of "valid" configurations

220

In [7]:
aoc.submitPartOne(220)

Your answer: 220. 
 That's the right answer! You are one gold star closer to finding the Chief Historian. [Continue to Part Two]

In [8]:
aoc.viewPartTwo()

--- Part Two --- 
 The engineers are surprised by the low number of safe reports until they realize they forgot to tell you about the Problem Dampener . 
 The Problem Dampener is a reactor-mounted module that lets the reactor safety systems tolerate a single bad level in what would otherwise be a safe report. It's like the bad level never happened! 
 Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe. 
 More of the above example's reports are now safe: 
 
 7 6 4 2 1 : Safe without removing any level. 
 1 2 7 8 9 : Unsafe regardless of which level is removed. 
 9 7 6 2 1 : Unsafe regardless of which level is removed. 
 1 3 2 4 5 : Safe by removing the second level, 3 . 
 8 6 4 4 1 : Safe by removing the third level, 4 . 
 1 3 6 7 9 : Safe without removing any level. 
 
 Thanks to the Problem Dampener, 4 reports are actually safe ! 
 Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports. How many reports are now safe?

In [10]:
fun delta(pair: Pair<Int, Int>) = pair.first - pair.second
fun neighboredDelta(report: List<Int>) = report.zipWithNext().map(::delta)
fun List<List<Int>>.filterInRange(lower: Int, upper: Int) = this.filter { it.all { abs(it) in lower..upper } }

aoc.input()
    .lines()
    .map { it.split(" ") }
    .map { it.map { it.toInt() } }
    .map { record -> List(record.size) { record.filterIndexed { i, _ -> i != it } } } // get all combinations of removing 1 level
    .map { record ->
        record // i can now feed an exact copy paste of part 1 to each of these combinations
            .filter { (it.sorted() == it) or (it.sortedDescending() == it) } 
            .map { (neighboredDelta(it) to neighboredDelta(it.reversed())) } 
            .unzip()
            .let { (forward, backward) -> forward.filterInRange(1, 3) to backward.filterInRange(1, 3) }
            .let { (forward, backward) -> min(forward.size, backward.size) }
            .sign // lastly, get the sign in order to count it or not (must be 0 or 1)
    }
    .sum() // get the total number of safe reports

296

In [11]:
aoc.submitPartTwo(296)

Your answer: 296. 
 That's the right answer! You are one gold star closer to finding the Chief Historian. 
 You have completed Day 2! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .